In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import re
import pickle as pk
from scipy.signal import hilbert
from statsmodels.tsa.seasonal import seasonal_decompose

In [4]:
path =  'd:/Git_Proyects/Tesis/Faiss_vect/'
citas = pk.load(open(path+'cita_media_cluster_october.pk', 'rb'))

In [5]:
# Fileid of the series to analyze
def load_files(index):
    citas = pk.load(open('d:/Git_Proyects/Tesis/Faiss_vect/cita_media_cluster_october.pk', 'rb'))
    filename = f'd:/Git_Proyects/Tesis/Faiss_vect/Tw_mas_RT_October/RT_Tw_cluster_{index}.csv'
    df = pd.read_csv(filename)
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%a %b %d %H:%M:%S +0000 %Y')

    categoria = 'RT'
    df_categoria_true = df[df[categoria] == True]
    df_categoria_false = df[df[categoria] == False]
    df_categoria = [df_categoria_false, df_categoria_true]
    max_date= []
    trendss = []
    df_crudo = []
    for i, df1 in enumerate(df_categoria):
        df_index = df1.set_index('Fecha')
        df_count_hora = df_index.resample('1H').count()
        df_crudo.append(df_count_hora)
        df_categoria = [df_categoria_true, df_categoria_false]
        result = seasonal_decompose(df_count_hora['Tweets'], model='additive')
        trend = result.trend.fillna(0)

        fecha_max_apariciones = trend.idxmax()
        max_date.append(fecha_max_apariciones)
        fecha_inicio = max_date[0] - pd.Timedelta(hours=1)
        fecha_fin = max_date[0] + pd.Timedelta(days=7)
        trednd_max = trend.loc[str(fecha_inicio):str(fecha_fin)]

        trendss.append(trednd_max)
    d1 = {'time': trendss[1].index, 'blog':trendss[1].values/max(trendss[1].values)}
    d2 = {'time': trendss[0].index, 'media':trendss[0].values/max(trendss[0].values)}

    dfRT = pd.DataFrame(d1)
    dfT = pd.DataFrame(d2)

    data = dfRT.merge(dfT, on='time')
    data.set_index('time', inplace=True)
    data.media = data.media.div(data.media.max())
    data.blog = data.blog.div(data.blog.max())

    # Cambiar acá si queremos ajustar solo una parte de los datos
    init_index = data.media.argmax()
    data = data[init_index:init_index + 168]
    return data

def make_figure(data, index):
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize = (8,5), dpi = 100)
    ax.plot()
    #ax.plot(data_crudo.media.to_numpy(), 'o-', color = 'm', linewidth = 2, label = 'Tw crudo', alpha = 0.85)
    ax.plot(data.media.to_numpy(), '.-', color = 'red', linewidth = 2, label = 'Tw suavizado', alpha =1)

    ax.set_ylim([0.00, 1.05])
    ax.set_xlim([0, data.shape[0]])
    ax.grid(True, alpha = 0.15)
    #ax.set_title('Media')

    #ax.plot(data_crudo.blog.to_numpy(), 'o-', color = 'c', linewidth = 2, label = 'RT crudo', alpha = 0.85)
    ax.plot(data.blog.to_numpy(), '.-', color = 'blue', linewidth = 2, label = 'RT suavizado', alpha = 1)
    ax.set_ylim([0.00, 1.05])
    ax.set_xlim([0, data.shape[0]])
    ax.grid(True, alpha = 0.15)
    #ax.set_title(citas[index]+ '\nTw y RT')

    fig.tight_layout()
    plt.legend()
    plt.grid(alpha = 0.7)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)

    plt.xlabel('Tiempo [horas]', fontsize = 14)
    plt.savefig(path+f'Cuenta_imagenes/serie_{index}_oct.pdf')
    plt.close()


In [6]:
mes = 'october'
archivos = os.listdir(path+f'Tw_mas_RT_{mes}')

for archivo in archivos:
    
    patron = re.compile(r'_(\d+).csv')
    coincidencia = patron.search(archivo)
    index = coincidencia.group(1)


    d = load_files(index)
    make_figure(d, index)

C:\Users\Ale\AppData\Local\Temp\ipykernel_2132\386516621.py:30: RuntimeWarning: invalid value encountered in divide
  d1 = {'time': trendss[1].index, 'blog':trendss[1].values/max(trendss[1].values)}
